In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the path to your dataset
Datadir = "C:\\Users\\shrey\\Downloads\\datasets\\train"

# Define the list of emotion categories
Classes = ["surprise","sad","neutral","happy","fear","disgust","angry"]  # Add more classes if needed

# Function to load and preprocess the dataset
def load_dataset():
    data = []
    labels = []

    for category in Classes:
        path = os.path.join(Datadir, category)
        class_num = Classes.index(category)

        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                img_arr = cv2.resize(img_arr, (50, 50))  # Adjust the size as needed
                data.append(img_arr)
                labels.append(class_num)
            except Exception as e:
                print(f"Error reading image: {os.path.join(path, img)}")

    data = np.array(data)
    labels = np.array(labels)

    return data, labels


In [3]:
print(len(data))

28680


In [2]:

# Load the dataset
data, labels = load_dataset()

# Split the data into training and testing sets (80% training, 20% testing)
split_ratio = 0.8
split_index = int(len(data) * split_ratio)

X_train, y_train = data[:split_index], labels[:split_index]
X_test, y_test = data[split_index:], labels[split_index:]

# Normalize the pixel values to be between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(50, 50, 1)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(Classes), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train.reshape(-1, 50, 50, 1), y_train, epochs=20, validation_data=(X_test.reshape(-1, 50, 50, 1), y_test))

Epoch 1/20
717/717 [==============================] - 112s 154ms/step - loss: 1.3206 - accuracy: 0.4604 - val_loss: 8.3643 - val_accuracy: 0.0204
Epoch 2/20
717/717 [==============================] - 116s 162ms/step - loss: 1.1193 - accuracy: 0.5573 - val_loss: 9.9326 - val_accuracy: 0.0282
Epoch 3/20
717/717 [==============================] - 118s 164ms/step - loss: 1.0198 - accuracy: 0.6000 - val_loss: 9.4135 - val_accuracy: 0.0333
Epoch 4/20
717/717 [==============================] - 111s 154ms/step - loss: 0.9288 - accuracy: 0.6379 - val_loss: 11.4173 - val_accuracy: 0.0486
Epoch 5/20
717/717 [==============================] - 119s 165ms/step - loss: 0.8345 - accuracy: 0.6815 - val_loss: 12.3982 - val_accuracy: 0.0579
Epoch 6/20
717/717 [==============================] - 115s 160ms/step - loss: 0.7273 - accuracy: 0.7240 - val_loss: 11.9119 - val_accuracy: 0.0833
Epoch 7/20
717/717 [==============================] - 117s 164ms/step - loss: 0.6136 - accuracy: 0.7725 - val_loss: 14.13

In [4]:
model.save("my_model_1.keras")
model= tf.keras.models.load_model("my_model_1.keras")

In [1]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("my_model_1.keras")

# Load the pre-trained Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create a list of emotion labels
emotion_labels = ["surprise","sad","neutral","happy","fear","disgust","angry"]

#Video:
# video_path ="C:\\Users\\shrey\\OneDrive\\Desktop\\playing-boy-happy-boy-together-portrait.jpg"
cap = cv2.VideoCapture(0)

while True:
    # Capture a single frame
    ret, frame = cap.read()

    if not ret:
        print("Video ended or cannot be opened.")
        break

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = gray_frame[y:y + h, x:x + w]

        # Resize the face region to match the input size of the model
        resized_face = cv2.resize(face_roi, (50, 50))

        # Normalize the pixel values to be between 0 and 1
        normalized_face = resized_face / 255.0

        # Reshape the face to match the input shape expected by the model
        input_face = normalized_face.reshape(-1, 50, 50, 1)

        # Make a prediction using the trained model
        predictions = model.predict(input_face)

        # Get the index of the predicted emotion
        predicted_emotion_index = np.argmax(predictions)

        # Get the corresponding emotion label
        facial_expression = emotion_labels[predicted_emotion_index]

        # Define behaviors based on predicted emotions
        if facial_expression in ['happy', 'neutral']:
            behavior = 'Engaged'
        elif facial_expression in ['sad', 'angry']:
            behavior = 'Distressed'
        elif facial_expression == 'surprise':
            behavior = 'Curious'
        elif facial_expression == 'fear':
            behavior = 'Anxious'
        elif facial_expression == 'disgust':
            behavior = 'Uncomfortable'
        else:
            behavior = 'Uncertain' 
        

    
 

# Main loop to capture video frames and c
        # Display the predicted emotion and behavior on the frame
        cv2.putText(frame, f"Emotion: {facial_expression}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f"Behavior: {behavior}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Facial Expression Recognition', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video file and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step


In [4]:
from tensorflow.keras.models import load_model

# Replace 'your_model_path' with the actual path to your pre-trained model
model = load_model('my_model_1.keras')
from tensorflow.keras.preprocessing import image
import numpy as np

from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(50, 50), color_mode='grayscale')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_emotion(model, img_array):
    prediction = model.predict(img_array)
    emotion_labels = ["surprise","sad","neutral","happy","fear","disgust","angry"]
    predicted_emotion = emotion_labels[np.argmax(prediction)]
    return predicted_emotion
# Example usage
image_path =r"C:\Users\shrey\OneDrive\Desktop\project student\cc_istock_78822855_large-2_16x9.jpg"
img_array = preprocess_image(image_path)
predicted_emotion = predict_emotion(model, img_array)
print(f'The predicted emotion is: {predicted_emotion}')

1/1 [==============================] - 0s 131ms/step
The predicted emotion is: surprise


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("my_model_1.keras")

# Load the pre-trained Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create a list of emotion labels
emotion_labels = ["surprise","sad","neutral","happy","fear","disgust","angry"]

# Load the image
# image_path = "C:\\Users\\shrey\\OneDrive\\Desktop\\360_F_321726194_pHtxATrvAwNke9rwEEmUDVNXBIRJGefv.jpg"  # Replace with your image file path
frame = cv2.imread(image_path)

# Convert the frame to grayscale
gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Detect faces in the frame
faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

for (x, y, w, h) in faces:
    # Extract the face region
    face_roi = gray_frame[y:y + h, x:x + w]

    # Resize the face region to match the input size of the model
    resized_face = cv2.resize(face_roi, (50, 50))

    # Normalize the pixel values to be between 0 and 1
    normalized_face = resized_face / 255.0

    # Reshape the face to match the input shape expected by the model
    input_face = normalized_face.reshape(-1, 50, 50, 1)

    # Make a prediction using the trained model
    predictions = model.predict(input_face)

    # Get the index of the predicted emotion
    predicted_emotion_index = np.argmax(predictions)

    # Get the corresponding emotion label
    facial_expression = emotion_labels[predicted_emotion_index]

    # Define behaviors based on predicted emotions
    if facial_expression in ['happy', 'neutral']:
        behavior = 'Engaged'
    elif facial_expression in ['sad', 'angry']:
        behavior = 'Distressed'
    elif facial_expression == 'surprise':
        behavior = 'Curious'
    elif facial_expression == 'fear':
        behavior = 'Anxious'
    elif facial_expression == 'disgust':
        behavior = 'Uncomfortable'
    else:
        behavior = 'Uncertain'

    # Display the predicted emotion and behavior on the frame
    cv2.putText(frame, f"Emotion: {facial_expression}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Behavior: {behavior}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
  

    # Draw a rectangle around the detected face
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

# Display the frame
cv2.imshow('Facial Expression Recognition', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()